In [1]:
import imagingcontrol4 as ic4
ic4.Library.init()

In [2]:
def print_device_list():
    print("Enumerating all attached video capture devices...")

    device_list = ic4.DeviceEnum.devices()

    if len(device_list) == 0:
        print("No devices found")
        return

    print(f"Found {len(device_list)} devices:")

    for device_info in device_list:
        print(format_device_info(device_info))

def format_device_info(device_info: ic4.DeviceInfo) -> str:
    return f"Model: {device_info.model_name} Serial: {device_info.serial}"

In [3]:
print_device_list()

Enumerating all attached video capture devices...
Found 1 devices:
Model: DFK 33UX183 Serial: 10524129


In [4]:
def print_interface_device_tree():
    print("Enumerating video capture devices by interface...")

    interface_list = ic4.DeviceEnum.interfaces()

    if len(interface_list) == 0:
        print("No interfaces found")
        return

    for itf in interface_list:
        print(f"Interface: {itf.display_name}")
        print(f"\tProvided by {itf.transport_layer_name} [TLType: {str(itf.transport_layer_type)}]")

        device_list = itf.devices

        if len(device_list) == 0:
            print("\tNo devices found")
            continue

        print(f"\tFound {len(device_list)} devices:")

        for device_info in device_list:
            print(f"\t\t{format_device_info(device_info)}")

In [5]:
print_interface_device_tree()

Enumerating video capture devices by interface...
Interface: Local Area Connection* 10 Microsoft Wi-Fi Direct Virtual Adapter #2
	Provided by IC4 GenTL Driver for GigEVision Devices [TLType: 1]
	No devices found
Interface: Ethernet Intel(R) Ethernet Controller I226-V
	Provided by IC4 GenTL Driver for GigEVision Devices [TLType: 1]
	No devices found
Interface: vEthernet (Default Switch) Hyper-V Virtual Ethernet Adapter
	Provided by IC4 GenTL Driver for GigEVision Devices [TLType: 1]
	No devices found
Interface: Intel(R) USB 3.20 eXtensible Host Controller - 1.20 (Microsoft)
	Provided by IC4 GenTL Driver for USB3Vision Devices [TLType: 2]
	Found 1 devices:
		Model: DFK 33UX183 Serial: 10524129


In [6]:
def handle_device_list_changed(device_enum: ic4.DeviceEnum):
    print("Device list changed!")

    print(f"Found {len(ic4.DeviceEnum.devices())} devices")

    print(ic4.DeviceEnum.devices())

def example_device_list_changed():
    enumerator = ic4.DeviceEnum()

    token = enumerator.event_add_device_list_changed(handle_device_list_changed)

    print("Waiting for DeviceListChanged event")
    print("Press Enter to exit")
    input()

    # Technically, this is not necessary, because the enumerator object is deleted when the function is exited
    # But for demonstration purposes, the event handler is removed:
    enumerator.event_remove_device_list_changed(token)

In [7]:
# Create a Grabber object
grabber = ic4.Grabber()

# Open the first available video capture device
first_device_info = ic4.DeviceEnum.devices()[0]
grabber.device_open(first_device_info)

In [8]:
# Configure the device to output images in the Mono8 pixel format
grabber.device_property_map.set_value(ic4.PropId.PIXEL_FORMAT, ic4.PixelFormat.Mono8)

# Set the resolution to 640x480
grabber.device_property_map.set_value(ic4.PropId.WIDTH, 640)
grabber.device_property_map.set_value(ic4.PropId.HEIGHT, 480)

# Set the origin of the ROI to the top-left corner of the sensor
grabber.device_property_map.set_value(ic4.PropId.OFFSET_AUTO_CENTER, "Off")
grabber.device_property_map.set_value(ic4.PropId.OFFSET_X, 0)
grabber.device_property_map.set_value(ic4.PropId.OFFSET_Y, 0)

# Configure the exposure time to 5ms (5000µs)
grabber.device_property_map.set_value(ic4.PropId.EXPOSURE_AUTO, "Off")
grabber.device_property_map.set_value(ic4.PropId.EXPOSURE_TIME, 5000.0)

# Enable GainAuto
grabber.device_property_map.set_value(ic4.PropId.GAIN_AUTO, "Continuous")

In [9]:
# Create a SnapSink. A SnapSink allows grabbing single images (or image sequences) out of a data stream.
sink = ic4.SnapSink()
# Setup data stream from the video capture device to the sink and start image acquisition.
grabber.stream_setup(sink, setup_option=ic4.StreamSetupOption.ACQUISITION_START)

In [10]:
try:
    # Grab a single image out of the data stream.
    image = sink.snap_single(1000)

    # Print image information.
    print(f"Received an image. ImageType: {image.image_type}")

    # Save the image.
    image.save_as_bmp("test2.bmp")

except ic4.IC4Exception as ex:
    print(ex.message)

Received an image. ImageType: ImageType(pixel_format=<PixelFormat.Mono8: 17301505>, width=640, height=480)


In [11]:
print("Device open:", grabber.is_device_open)
print("Device valid:", grabber.is_device_valid)
print("Streaming:", grabber.is_streaming)
print("Acquisition active:", grabber.is_acquisition_active)

Device open: True
Device valid: True
Streaming: True
Acquisition active: True


In [12]:
# 4) Record 3s @30fps
writer = ic4.VideoWriter(ic4.VideoWriterType.MP4_H264)
itype = ic4.ImageType(ic4.PixelFormat.Mono8,
                      grabber.device_property_map.get_value_int(ic4.PropId.WIDTH),
                      grabber.device_property_map.get_value_int(ic4.PropId.HEIGHT))
writer.begin_file("clip.mp4", itype, frame_rate=30.0)
seq = sink.snap_sequence(90, 4000)
for b in seq:
    writer.add_frame_copy(b)
    b.release()
writer.finish_file()

grabber.stream_stop()

In [ ]:
class OpenCVLiveListener(ic4.QueueSinkListener):
    def __init__(self, display: ic4.Display):
        self.display = display
    
    def sink_connected(self, sink, image_type, min_buffers_required):
        return True
    
    def frames_queued(self, sink):
        buffer = sink.pop_output_buffer()
        
        # Chuyển buffer sang numpy array để xử lý với OpenCV
        buffer_wrap = buffer.numpy_wrap()
        
        # Xử lý ảnh với OpenCV (ví dụ: làm mờ)
        cv2.blur(buffer_wrap, (31, 31), buffer_wrap)
        
        # Thêm text lên ảnh
        cv2.putText(buffer_wrap, "Live Stream", (100, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Hiển thị frame đã xử lý
        self.display.display_buffer(buffer)

# Tạo sink với định dạng BGR8 cho OpenCV
sink = ic4.QueueSink(listener, [ic4.PixelFormat.BGR8], max_output_buffers=1)
grabber.stream_setup(sink)